## Test AI Angent Work flow for Bank Statements

In [1]:
import os, json, time, gc

from dotenv import load_dotenv
from IPython.display import HTML, Markdown, Image, Video
from tqdm import tqdm
from openai import OpenAI, AsyncOpenAI
import asyncio
import aiohttp
import pandas as pd
import torch
from ctypes import *

#fix bug with aysncio and jupyter
import nest_asyncio # for langchain async 
nest_asyncio.apply()

import numpy as np


# load environment variables from .env file
load_dotenv()

/home/johnsonhk88/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


True

In [2]:
class CFG:
    


    USE_OPIK = True#False
    OFFLINE = False #True # for Test offline environment
    USE_LLAMA3 = False # 
    USE_GEMMA2 = False # 
    USE_QWEN = False # 
    USE_DEEPSEEK = True # 
    USE_DEEPSCALE = False # 

    TASK_GEN = True # for generative Text output task (suitable for RAG project)
    TEST_LLM = True
    # Set LLM host framework
    USE_OLLAMA = True # for OLLAMA local LLM framework
    USE_OPENROUTER = False # for OpenRouter LLM framework
    USE_OPENAI = False # for OpenAI LLM framework
    USE_HUGGINGFACE = True # Pull model from Huggingface model hub
    USE_LMSTUIDO = False # for local LLM framework  
    USE_VLLM = False # for VLLM  LLM framework

    # mulitlingual LLM model 
    model1 = "meta-llama/Llama-3.2-3B-Instruct"  # llama3.2  3B-Instruct

    model2 =  "google/gemma-2-2b-it" # gemma 2 9B (mulitlingual)
    model3 = "Qwen/Qwen2.5-3B-Instruct" # Qwen 3B (mulitlingual)
    model4 = 'Qwen/Qwen2.5-7B-Instruct' # Qwen 7B (mulitlingual)
    model5 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B" # DeepSeek Distill 1.5B (mulitlingual)
    model6 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B" # DeepSeek Distill 7B (mulitlingual)
    model7 = "agentica-org/DeepScaleR-1.5B-Preview"

    # for VLM model
    vlmModel1 = "Qwen/Qwen2.5-VL-3B-Instruct"
    vlmModel2 = "Qwen/Qwen2.5-VL-7B-Instruct"


    # Mult Embedding model
    embedModel1 = "intfloat/multilingual-e5-small-instruct" 
    embedModel2 = "intfloat/multilingual-e5-large-instruct"
    embedModel3 = "Alibaba-NLP/gte-Qwen2-1.5B-instruct" # for embedding model support chinese
    embedModel4 = "Alibaba-NLP/gte-multilingual-base" # for embedding model support chinese
    embedModel5 = "BAAI/bge-m3" # for multilingual embedding model
    embedModel6 = "jinaai/jina-embeddings-v3"
    embedModel7 = "ollama/mxbai-embed-large"  # ollama 
    embedModel8 = "ollama/nomic-embed-text"  # ollama 
    

    # LLM for AI Agent  OLLAMA (for local deployment) ,requires pulling the ollama image and running the server locally
    llmModel1 = "ollama/deepseek-r1:8b"
    llmModel2 = "ollama/deepseek-r1:7b"
    llmModel3 = "ollama/qwen2.5" # Qwen 2.5 7B
    llmModel4 = "ollama/llama3.2"
    llm_base_url1 = "http://localhost:11434"

    #LLM for AI Agent Openrouter 
    llmModel5 = "openrouter/qwen/qwen3-32b:free" # for openrouter
    llmModel6 = "openrouter/google/gemini-2.0-flash-exp:free"
    llm_base_url2="https://openrouter.ai/api/v1"
    api_key2 = os.getenv("OPENROUTER_API_KEY")
   
    # 



    FEW_SHOT_TEST= False#True
    USE_WANDB = True#True # for  LLM evalution and debug , track fine tuning performance

    USE_DEEPEVAL = True#False # for LLM evalution   
    USE_TRAIN =  False #True #False#True Much be use GPU for Training 
    
    # For VectorDB selection
    USE_FAISS = False#True # For RAG VectorDB
    USE_CHROMA = True #False #True #False # for RAG VectorDF
    USE_PINECONE = False#True#False #True # for RAG VectorDF
    USE_WEAVIATE = False#True #False # for RAG VectorDF
    USE_MILVUS = False#True              # for RAG VectorDF

    # for LLM fine tuning
    maxTrainData = 200#3500#5000 #10000#5000 #10000
    maxEvalData = 20#100 # 20 


    # LLM parameters
    reportTo ="none"
    topK = 40
    topP = 1.0
    temperature = 0.6 #0.5
    repetition_penalty = 1.05 # 1.1
    maxOutToken = 1024#180 #100
    

    
    maxToken=  512#768#512#768 # 512 for test only

In [3]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai.tools import BaseTool, tool
from crewai import LLM
import crewai
from pydantic import BaseModel, Field
from typing import List, Dict, Type
from crewai_tools import PDFSearchTool

/home/johnsonhk88/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/lib/python3/dist-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 45.0.0.
  "class": algorithms.Blowfish,
/usr/lib/python3/dist-packages/paramiko/tra

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
from langchain_ollama import OllamaLLM, ChatOllama
from langchain_openai import ChatOpenAI , OpenAIEmbeddings
from openai import OpenAI, AsyncOpenAI


In [6]:
import ctypes
def clearMemory():
    for _ in range(5):
        torch.cuda.empty_cache()
        ctypes.CDLL("libc.so.6").malloc_trim(0)
        gc.collect()
        time.sleep(0.3)

In [7]:
# clearMemory()

In [8]:
# !pip3 install --upgrade pip
# !pip3 install -qU langchain-qdrant
# !pip3 install -qU langchain-ollama

In [9]:
## Define Test Document path 
pdfFilePath1 = "../test-document/Attention .pdf"
pdfFilePath2 = "../test-document/yolo.pdf"
pdfDir = "../test-document"

bankStatementDir = "../bank-statement-document/"
bankStatementSamples =  "../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf"

## CrewAI Support LLM List :  LiteLLM 
- <https://docs.crewai.com/how-to/llm-connections>
- <https://docs.crewai.com/concepts/llms>



In [10]:
from crewai_tools import PDFSearchTool, YoutubeChannelSearchTool
from crewai_tools import RagTool

from crewai.tools import BaseTool, tool

# Chanin
from langchain_community.document_loaders import (TextLoader,
                                                  PyMuPDFLoader,
                                                  PyPDFDirectoryLoader,
                                                  PyPDFLoader)
from langchain_core.documents import Document

In [11]:
# # for ML Model
# if CFG.USE_WANDB:
#     # train report to  W&B tool
#     # import wandb
#     # reportTo= "wandb"
#     # my_secret = os.getenv("wandb_api_key") 
#     # wandb.login(key=my_secret) # login 
#     # wandb.init(
# else: 
#     reportTo = "none"# None

if CFG.USE_WANDB:
    import wandb
    # reportTo= "wandb"
    my_secret = os.getenv("wandb_api_key") 
    wandb.login(key=my_secret) # login 
    import weave
    # Initialize Weave with your project name 
    weave.init("crewai-ai-bank-statement-document-analysis-project")




/home/johnsonhk88/.local/lib/python3.10/site-packages/jupyter_events/validators.py:4: DeprecationWarning: jsonschema.RefResolver is deprecated as of v4.18.0, in favor of the https://github.com/python-jsonschema/referencing library, which provides more compliant referencing behavior as well as more flexible APIs for customization. A future release will remove RefResolver. Please file a feature request (on referencing) if you are missing an API for the kind of customization you need.
  from jsonschema import Draft7Validator, RefResolver, ValidationError
2025-06-11 21:53:22,515 - 129668737216512 - jupyter.py-jupyter:224 - ERROR: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: johnsonhk88. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the W

weave version 0.51.52 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: johnsonhk88.
View Weave data at https://wandb.ai/johnsonhk88/crewai-ai-bank-statement-document-analysis-project/weave


In [12]:
import opik
from opik import track
from opik.integrations.crewai import track_crewai
# for LiteLLM in crew 
from litellm.integrations.opik.opik import OpikLogger
import litellm

if CFG.USE_OPIK:
    opik.configure(use_local=True , automatic_approvals=True , url="http://localhost:5173")

    track_crewai(project_name="crewai-ai-bank-statement-document-analysis-project")
    opik_logger = OpikLogger()
    litellm.callbacks = [opik_logger]

### test pdf document loader

In [13]:


loader = PyMuPDFLoader(bankStatementSamples, extract_tables="markdown")
documents = loader.load()
docs= []
for doc in documents:
    # print(doc)
    print("=====================================")
    print(f"page content: {doc.page_content}")
    print(f"metaData : {doc.metadata}")
    print("=====================================")
    # print
    temp= {
        "page_content": doc.page_content,
        "metadata": doc.metadata

    }
    docs.append(temp)
df = pd.DataFrame(docs , columns=["page_content", "metadata"])

page content: Issue Date:
Period:
Account Activity
Date
Payment Type
Paid In
Paid Out
Balance
Your Account Statement
Detail
Note:
Print Form
Save Form
Reset Form
<Branch Name>
231 Valley Farms Street 
Santa Monica, CA 
bickslowbank@domain.com
mm/dd/yyyy
mm/dd/yyyy to mm/dd/yyyy
111-234-567-890  
Bit Manufacturing Ltd
2450 Courage St, STE 108
Brownsville, TX 78521
Balance Brought Forward
8,313.30
mm/dd/yyyy Fast Payment
Amazon
132.30
8,181.00
mm/dd/yyyy BACS
eBAY Trading Co.
515.22
7,665.78
mm/dd/yyyy Fast Payment
Morrisons Petrol
80.00
7,585.78
mm/dd/yyyy BACS
Business Loan
20,000.00
27,585.78
mm/dd/yyyy BACS
Jumes White Media
2,416.85
25,168.93
mm/dd/yyyy Fast Payment
ATM High Street
100.00
25,068.93
mm/dd/yyyy BACS
Accorn Advertising Studios
150.00
24,918.93
Fast Payment
mm/dd/yyyy
Marriott Hotels
177.00
24,741.93
mm/dd/yyyy Fast Payment
Abelio Scotrail Ltd
122.22
24,619.71
mm/dd/yyyy Fast Payment
Cheque 000234
1,200.00
23,419.71
mm/dd/yyyy Int. Bank
Interest Paid
9.33
23,429.04
mm/d

<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


In [14]:
# !pip3 list

In [15]:
# !pip freeze > new-requirements.txt

In [16]:
# df

In [17]:
tempDict = df.to_dict(orient="records")
tempDict[0]

{'page_content': 'Issue Date:\nPeriod:\nAccount Activity\nDate\nPayment Type\nPaid In\nPaid Out\nBalance\nYour Account Statement\nDetail\nNote:\nPrint Form\nSave Form\nReset Form\n<Branch Name>\n231 Valley Farms Street \nSanta Monica, CA \nbickslowbank@domain.com\nmm/dd/yyyy\nmm/dd/yyyy to mm/dd/yyyy\n111-234-567-890  \nBit Manufacturing Ltd\n2450 Courage St, STE 108\nBrownsville, TX 78521\nBalance Brought Forward\n8,313.30\nmm/dd/yyyy Fast Payment\nAmazon\n132.30\n8,181.00\nmm/dd/yyyy BACS\neBAY Trading Co.\n515.22\n7,665.78\nmm/dd/yyyy Fast Payment\nMorrisons Petrol\n80.00\n7,585.78\nmm/dd/yyyy BACS\nBusiness Loan\n20,000.00\n27,585.78\nmm/dd/yyyy BACS\nJumes White Media\n2,416.85\n25,168.93\nmm/dd/yyyy Fast Payment\nATM High Street\n100.00\n25,068.93\nmm/dd/yyyy BACS\nAccorn Advertising Studios\n150.00\n24,918.93\nFast Payment\nmm/dd/yyyy\nMarriott Hotels\n177.00\n24,741.93\nmm/dd/yyyy Fast Payment\nAbelio Scotrail Ltd\n122.22\n24,619.71\nmm/dd/yyyy Fast Payment\nCheque 000234\n1,20

In [18]:
#
def convertDataFrameToDocs(df):
    """ Convert DataFrame to list of LangChain Documents"""
    docs = []
    tempDict = df.to_dict(orient="records")
    for temp in tempDict:
        document = Document(
            page_content=temp["page_content"],
            metadata=temp["metadata"]
        )
        docs.append(document)
    # document = Document(
    #     page_content=tempDict[0]["page_content"],
    #     metadata=tempDict[0]["metadata"]
    # )
    return docs

In [19]:
# Convert DataFrame to LangChain Document
pdfDocs = convertDataFrameToDocs(df)

In [20]:
# save the document to Vector Database
pdfDocs

[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': '', 'creationdate': '2020-07-03T16:22:11+08:00', 'source': '../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf', 'file_path': '../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': 'Bank Statement Template 2 - TemplateLab.xlsx', 'author': 'HFO Desktop', 'subject': '', 'keywords': '', 'moddate': '2020-07-03T19:36:51+08:00', 'trapped': '', 'modDate': "D:20200703193651+08'00'", 'creationDate': "D:20200703162211+08'00'", 'page': 0}, page_content='Issue Date:\nPeriod:\nAccount Activity\nDate\nPayment Type\nPaid In\nPaid Out\nBalance\nYour Account Statement\nDetail\nNote:\nPrint Form\nSave Form\nReset Form\n<Branch Name>\n231 Valley Farms Street \nSanta Monica, CA \nbickslowbank@domain.com\nmm/dd/yyyy\nmm/dd/yyyy to mm/dd/yyyy\n111-234-567-890  \nBit Manufacturing Ltd\n2450 Courage St, STE 108\nBrownsville, TX 78521\nBalance Brought Forward\

# Embedding Model

In [21]:
# from langchain.vectorstores import Qdrant
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

from sentence_transformers import SentenceTransformer 

from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma #
from langchain_openai import OpenAIEmbeddings, AzureOpenAIEmbeddings
from openai import OpenAI
from langchain_ollama import OllamaEmbeddings

# 
from langchain_text_splitters import RecursiveCharacterTextSplitter


/home/johnsonhk88/.local/lib/python3.10/site-packages/langchain_huggingface/chat_models/__init__.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_huggingface.chat_models.huggingface import (
/home/johnsonhk88/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.p

In [22]:
def HuggingFaceEmbeddingInit(modelName: str= CFG.embedModel1 , remote_code: bool = False):
    """ Initialize the HuggingFace Embedding model"""
    if device.type =="cpu":
        # model_kwargs = {'device': 'cpu', "torch_dtype": "float16"} # not support torch_dtype in Hugggiungface embeding at langchain
        model_kwargs = {'device': 'cpu', "trust_remote_code": remote_code}
    else :
        model_kwargs = {"device": "cuda", "trust_remote_code": remote_code}
    embedding_model = HuggingFaceEmbeddings(
        model_name=modelName,
        model_kwargs=model_kwargs
    )
    return embedding_model


def OllamaEmbeddingInit(modelName: str =CFG.embedModel8 , url=None):
    """ Initialize the Ollama Embedding model"""
    embedding_model = OllamaEmbeddings(
        model=modelName,
        base_url=url
    )
    return embedding_model

In [23]:
# Initialize the HuggingFace Embedding model
# embed  =HuggingFaceEmbeddingInit(modelName=CFG.embedModel3,  remote_code=True)
embed = OllamaEmbeddingInit(modelName="nomic-embed-text", url=None)
embed

OllamaEmbeddings(model='nomic-embed-text', base_url=None, client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, keep_alive=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [ ]:
# # Initialize Qdrant vector database
# qdrant = Qdrant.from_documents(
#     documents=[pdfDocs],  # List of LangChain documents
#     embedding=embed,
#     location=":memory:",  # Use in-memory Qdrant instance; replace with actual Qdrant URL if needed
#     collection_name="bank_statements"
# )

# print("Document saved to Qdrant vector database.")

def initQdrantDB(embedding_model, collection_name: str = "bank_statements", embedding_dimension: int = 768):
    """ Initialize Qdrant vector database"""
    client = QdrantClient(":memory:")
    
    # Check if the collection already 
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=embedding_dimension, distance=Distance.COSINE),
    )

    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embedding_model,
    )
    # qdrant = QdrantClient(url=url)
    # qdrant.recreate_collection(
    #     collection_name=collection_name,
    #     vectors_config=VectorParams(
    #         size=embedding_model.embedding_dimension,
    #         distance=Distance.COSINE
    #     )
    # )
    return vector_store


def saveToVDB(db, chuck: list[Document]):
    startTime = time.time()
    ids = db.add_documents(chuck)
    print(f"Time Taken:  {time.time() - startTime}")
    return ids    


async def saveToVDBAsync(db, chuck: list[Document]):
    startTime = time.time()
    ids = await db.aadd_documents(chuck)
    print(f"Time Taken:  {time.time() - startTime}")
    return ids    


def vectorDBsimilaritySearchWithScores(db, query, k =3):
        results = db.similarity_search_with_relevance_scores(query, k=k)
        return results

def vectorDBsimilaritySearch(db, query, k=3):
        results = db.similarity_search(query, k=k)
        return results


def vectorDBsimilaritySearchByVector(db, query, embed, k=3 ):
        results = db.similarity_search_by_vector(
        embedding=embed.embed_query(query), k=k)
        return results

def vectorDBSimilaritySearchByMMR(db, query, k=3, fetchK=20):
        results = db.max_marginal_relevance_search(query=query, k=k, fetch_k=fetchK)
        return results


In [25]:



def textSplitterByText(txt, chunkSize= 512, overlap= 20, separators=["\n\n"]):
    textSplitter = RecursiveCharacterTextSplitter(
        chunk_size = chunkSize,
        chunk_overlap = overlap,
        separators=separators,
        length_function = len,
        is_separator_regex =False
    )
    splittedTxt = textSplitter.split_text(txt)
    return splittedTxt

def textSplitterByDocs(docs, chunkSize= 512, overlap= 20, separators=["\n\n"]):
    textSplitter = RecursiveCharacterTextSplitter(
        chunk_size = chunkSize,
        chunk_overlap = overlap,
        separators=separators,
        length_function = len,
        is_separator_regex =False
    )
    splitted = textSplitter.split_documents(docs)
    return splitted


In [26]:
vectDB = initQdrantDB(embed, collection_name="bank_statements", embedding_dimension=768)

In [27]:
splittedDocs = textSplitterByDocs(pdfDocs, chunkSize=512, overlap=20, separators=["\n\n"])

In [28]:
# splittedDocs[].metadata

In [29]:
ids = await saveToVDBAsync(vectDB, splittedDocs)

Time Taken:  0.03516101837158203


In [30]:
ids

['b98eb3fd13fe461b8256065f03646939']

In [31]:
### Test Data Retrieval from Vector Database

In [32]:
res =vectDB.similarity_search("What is the total amount in the bank statement?", k=3)

In [33]:
res , len(res)

([Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': '', 'creationdate': '2020-07-03T16:22:11+08:00', 'source': '../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf', 'file_path': '../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': 'Bank Statement Template 2 - TemplateLab.xlsx', 'author': 'HFO Desktop', 'subject': '', 'keywords': '', 'moddate': '2020-07-03T19:36:51+08:00', 'trapped': '', 'modDate': "D:20200703193651+08'00'", 'creationDate': "D:20200703162211+08'00'", 'page': 0, '_id': 'b98eb3fd13fe461b8256065f03646939', '_collection_name': 'bank_statements'}, page_content='Issue Date:\nPeriod:\nAccount Activity\nDate\nPayment Type\nPaid In\nPaid Out\nBalance\nYour Account Statement\nDetail\nNote:\nPrint Form\nSave Form\nReset Form\n<Branch Name>\n231 Valley Farms Street \nSanta Monica, CA \nbickslowbank@domain.com\nmm/dd/yyyy\nmm/dd/yyyy to mm/dd/yyyy\n111-234-567-890  \nBit Manufactu

In [34]:
for val in res:
    print(f"* {val.page_content}\nMetaData[{val.metadata}]")

* Issue Date:
Period:
Account Activity
Date
Payment Type
Paid In
Paid Out
Balance
Your Account Statement
Detail
Note:
Print Form
Save Form
Reset Form
<Branch Name>
231 Valley Farms Street 
Santa Monica, CA 
bickslowbank@domain.com
mm/dd/yyyy
mm/dd/yyyy to mm/dd/yyyy
111-234-567-890  
Bit Manufacturing Ltd
2450 Courage St, STE 108
Brownsville, TX 78521
Balance Brought Forward
8,313.30
mm/dd/yyyy Fast Payment
Amazon
132.30
8,181.00
mm/dd/yyyy BACS
eBAY Trading Co.
515.22
7,665.78
mm/dd/yyyy Fast Payment
Morrisons Petrol
80.00
7,585.78
mm/dd/yyyy BACS
Business Loan
20,000.00
27,585.78
mm/dd/yyyy BACS
Jumes White Media
2,416.85
25,168.93
mm/dd/yyyy Fast Payment
ATM High Street
100.00
25,068.93
mm/dd/yyyy BACS
Accorn Advertising Studios
150.00
24,918.93
Fast Payment
mm/dd/yyyy
Marriott Hotels
177.00
24,741.93
mm/dd/yyyy Fast Payment
Abelio Scotrail Ltd
122.22
24,619.71
mm/dd/yyyy Fast Payment
Cheque 000234
1,200.00
23,419.71
mm/dd/yyyy Int. Bank
Interest Paid
9.33
23,429.04
mm/dd/yyyy DD
OV

# Define the tools for CrewAI agent

In [35]:
#Tool for AI agent

# pdfTool = PDFSearchTool(pdfDir=bankStatementDir)

In [36]:
# Custom PDF Search Tool for CrewAI
@tool("Custom PDF Extractor")
def pdf_extractor(
    pdf_dir: str
) -> Dict:
    """
    Extracts text from PDF files in the specified directory.
    Args:
        pdf_dir (str): The directory containing PDF files.
    Returns:
        List[Dict[str, str]]: A list of dictionaries containing the extracted text from each PDF file.
    """
    # results = pdfTool.search(query=query, pdf_dir=pdf_dir)
    loader = PyMuPDFLoader(pdf_dir)
    documents = loader.load() #return list of documents
    docs = []
    for doc in documents:
        # print(doc)
        print("=====================================")
        print(f"page content: {doc.page_content}")
        print(f"metaData : {doc.metadata}")
        print("=====================================")
        # print
        temp= {
            "page_content": doc.page_content,
            "metadata": doc.metadata

        }
        docs.append(temp)
    
    df = pd.DataFrame(docs , columns=["page_content", "metadata"])
    print(f"Extracted {len(df)} pages from PDF files in {pdf_dir}\n\r {type(df)}, {df}")
    return df

In [37]:
@tool("PDF directory Extractor")
def pdf_directory_extractor(
    pdf_dir: str = pdfDir
) -> list:
    """
    Extracts text from all PDF files in the specified directory.
    Parameters:
    - pdf_dir (str): The directory containing PDF files.
    Returns:
    - List of dictionaries containing the content and source of each PDF file.
    """
    loader = PyPDFDirectoryLoader(pdf_dir)
    documents = loader.load()
    
    # results = []
    # for doc in documents:
    #     if query.lower() in doc.page_content.lower():
    #         results.append({"content": doc.page_content, "source": doc.metadata.get("source", "")})
    
    # return results

    return documents

In [ ]:
@tool("Vector DB Store tool")
def vector_db_store_tool(
    doc: str
) -> list:
    """
    Store the extracted text from PDF files in a vector database.
    Parameters:
    - doc (str): The directory containing PDF files.
    Returns:
    - id (int) vector database idx 
    """
    # loader = PyPDFDirectoryLoader(pdf_dir)
    # documents = loader.load()
    print(f"Input document type: {type(doc)},  content : {doc}")

    #intitalize the embedding model
    collection_name: str = "bank_statements"
    embedding_dimension: int = 768
    embedding_model: str = CFG.embedModel1
    client = QdrantClient(":memory:")
    
    # Check if the collection already 
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=embedding_dimension, distance=Distance.COSINE),
    )

    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embedding_model,
    )

    # Convert the pdf document into chunks
    splittedDocs = textSplitterByDocs(doc, chunkSize=512, overlap=20, separators=["\n\n"])
    
    # Save the documents to the vector database

    # results = []
    # for doc in documents:
    #     if query.lower() in doc.page_content.lower():
    #         results.append({"content": doc.page_content, "source": doc.metadata.get("source", "")})
    
    # return results
    ids = saveToVDBAsync(vector_store, splittedDocs)
   

    return ids


In [ ]:
@tool("RAG Tool")
def rag_tool(query : str) -> str:
    """Tool to search 
    """
    #
    collection_name: str = "bank_statements"
    embedding_dimension: int = 768
    embedding_model: str = CFG.embedModel1
    client = QdrantClient(":memory:")
    
    # Check if the collection already 
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=embedding_dimension, distance=Distance.COSINE),
    )

    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embedding_model,
    )

    results = vectorDBsimilaritySearchByMMR(vector_store, query=query, k=3, fetchK=20)


    print(f"Search results for query '{query}':")

    # Print the results
    for res in results:
        print(f"- {res['metadata']['sentence']} (score: {res['score']})")

    return results

    # Encode the question using the embedding model
    # query_vec = model.encode(question)
    
    # # Get top 5 similar vector    # Encode the question
    # query_vec = model.encode(question)
    
    # # Get top 5 simil`ar vector
    # results = company_db.search(query_vec, top_k = 5)

    # # Build context from the results
    # context = "\n".join([f"- {res['metadata']['sentence']}" for res in results])

    # # Create the prompt
    # prompt = f"""You are a helpful assistant. Use the context below to answer the user's question.

    #         Context:
    #         {context}

    #         Question: {question}

    #         Answer:
    #         """

    # # Generate an answer using the context
    # client = OpenAI()

    # response = client.responses.create(
    #     model = "gpt-4o-mini",
    #     input = prompt
    # )

    # answer = response.output_text
    
    # # Return the answer
    # return answer


In [107]:
class DocumentExtractInput(BaseModel):
    query: str = Field(... , description="")


class DocumentSearchTool(BaseTool):
    name: str = "DocumentSearchTool"
    description: str = "Search the document for the given query."
    args_schema: Type[BaseModel] = DocumentExtractInput


    
    





In [108]:
class PDFSearchToolInput(BaseModel):
    query: str = Field(description="The search query.")

        
class PDFExtractTool(BaseTool):
    name : str =  "PDFSearchTool"
    description: str  = "A tool to search for information in PDF files."
    args_schema: Type[BaseModel] = PDFSearchToolInput
    
    # model_config = ConfigDict(
    #     arbitrary_types_allowed=True,
    # )

    def __init__(self, pdf_dir: str):
        """Initialize the PDF Search Tool."""
        super().__init__()
        self.pdf_dir = pdf_dir
        # initalize VectorDB 
        self.vectorDB = Chroma.from_documents(documents, embedding_function=embedModel1)
        # document process
        self._process_documents(self.pdf_dir)




    def _process_documents(self, pdf_dir: str):
        """Process the documents in the specified directory."""
        # loader = PyPDFDirectoryLoader(pdf_dir)
        # documents = loader.load()
        loader = PyMuPDFLoader(pdf_dir)
        documents = loader.load() # list of document 
        # 
        
        

        

    def _run(self, query: str) -> str:
        return "test"

In [109]:


if CFG.USE_OLLAMA:
    # # for ollama base LLM
    llm = LLM(
    # model= CFG.llmModel2,
    model= CFG.llmModel3,
    base_url= CFG.llm_base_url1,
    )
if CFG.USE_OPENROUTER:
    # for openrouter base LLM
    llm = LLM(
        model= CFG.llmModel6,
     base_url= CFG.llm_base_url2,
        api_key= CFG.api_key2
    )
# for openrouter base LLM
# 


## Define AI Agent

In [110]:
# 
document_agent = Agent( 
    role="Document Extract Specialist",
    goal="Extract text and structured data from PDF bank statements documents.",
    backstory="You are a document extraction agent. You will be given a document and you need to extract the relevant information from it.",
    # tools=[webSearchTool],
    tools=[pdf_extractor], # Custom PDF Search Tool]
    llm=llm,
    max_iter=3,
    max_execution_time=120, # seconds
    allow_delegation= False,
    verbose=True,

)


In [111]:
# data_processing_agent = Agent(
#   role= "Data Processor Specialist",
#   goal="Clean and process structured data from PDF bank statements documents.",
#   backstory="You are a data processing agent. You will be given a document and you need to clean and process the data.",
#   llm=llm,
#   max_iter=3,
#   max_execution_time=120, # seconds
#   allow_delegation= True,
#   verbose=True,

# )

In [112]:
vectordb_agent = Agent(
    role='VectorDB Manager',
    goal='Store vectorized financial data',
    backstory='Expert in vector storage',
    tools=[vector_db_store_tool], # 
    llm=llm,
    max_iter=3,
    max_execution_time=120, # seconds
    allow_delegation= True,
    verbose=True,

)

In [113]:
financial_analyst_agent = Agent(
    role='RAG Financial Analyst',
    goal= ('Analyze transaction data and identify key financial insights from VectorDB'
           "Use RAG to extract relevant information from the bank statement documents."
           ),
    backstory="Experienced financial analyst.",
    llm=llm,
    max_iter=3,
    max_execution_time=120, # seconds
    allow_delegation= True,
    verbose=True,
)

In [114]:
report_agent = Agent( 
    role= "Report Writter Expert for Financial Analystic",
    goal="Generate Clear and Concise Report and charts from the analysis results",
    backstory=("Professional report writer with expertise in financial analysis."),
    llm=llm,
    max_iter=3,
    max_execution_time=120, # seconds
    allow_delegation= True,
    verbose=True,
                     
                    
 )





# Task for AI Agent

In [115]:
load_document_task = Task(
    description="load the document from directory:  {pdf_dir} ",
    expected_output=("document content in JSON format with key: content, source , page_number"
                      "output the document content in list of PDF page content"
                      "Output JSON format of the extracted text from the PDF files in the directory with key: content, source , page_number"),
    agent=document_agent,  
    name="Load Document Task",
    verbose=True,
)

In [116]:
# process_task = Task(
#         description='Process and clean the extracted data', 
#         expected_output="document content in JSON format with key: content, source , page_number",
#         agent=data_processing_agent,
#         verbose=True,

# )

In [117]:
store_task = Task(
        description='Store data in VectorDB', 
        expected_output="document content in JSON format with key: content, source , page_number",
        agent=vectordb_agent,
        context=[load_document_task], #
        verbose=True

)

In [61]:
retrieve_analysis_task = Task(
    description="Retrieve the content from the document, ",
    expected_output="Output the content of the document relevant to the query",
    agent=financial_analyst_agent
)

In [118]:
Financial_Analytic_task = Task(
    description="Extract the financial information from the document, ",
    expected_output="Output the financial information from the document",
    agent=financial_analyst_agent,
)

In [63]:
report_task = Task(

    description="Generate a report from the document",
    expected_output="Output the report of the document",
    agent=report_agent
)

In [ ]:
crew1 = Crew(
    tasks=[
        load_document_task,
        store_task,
        # retrieve_analysis_task,
        Financial_Analytic_task,
        # summarize_task,
        # report_task
    ],
    agents=[
        document_agent,
        vectordb_agent,
        # data_structuring_agent,
        financial_analyst_agent,
        # rag_agent,
        # fiancial_agent,
        # summary_agent,
        # report_agent
    ],
    # max_concurrent_tasks=2, # limit the number of concurrent tasks
    verbose=True, 
)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (3746929039.py, line 12)

In [58]:
inputs = {
    "pdf_dir" : bankStatementSamples

}

In [59]:
result = await crew1.kickoff_async(inputs= inputs) 

/home/johnsonhk88/.local/lib/python3.10/site-packages/weave/trace_server/trace_server_interface.py:218: DeprecationWarning: deprecated
  if self.set_base_object_class is not None and self.builtin_object_class is None:


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1e4edb50-ccca-4d27-9c68-c01797a773cf                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/home/johnsonhk88/.local/lib/python3.10/site-packages/weave/trace_server/trace_server_interface.py:218: DeprecationWarning: deprecated
  if self.set_base_object_class is not None and self.builtin_object_class is None:


🚀 Crew: crew
└── 📋 Task: 5bcd43de-1593-4783-bca0-8ce802c1c743
       Status: Executing Task...

/home/johnsonhk88/.local/lib/python3.10/site-packages/weave/trace_server/trace_server_interface.py:218: DeprecationWarning: deprecated
  if self.set_base_object_class is not None and self.builtin_object_class is None:


...

🚀 Crew: crew
├── 📋 Task: 5bcd43de-1593-4783-bca0-8ce802c1c743
│      Assigned to: Document Extract Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Document Extract Specialist
│           Status: ✅ Completed
│       └── 🔧 Used Custom PDF Extractor (1)
└── 📋 Task: 6d3a286f-0f88-4b6f-bc85-59958f517592
       Assigned to: VectorDB Manager
       Status: ✅ Completed
    ├── 🤖 Agent: VectorDB Manager
    │       Status: In Progress
    │   ├── 🔧 Failed Vector DB Store tool (1)
    │   └── 🔧 Using Ask question to coworker (1)
    └── 🤖 Agent: VectorDB Manager
            Status: ✅ Completed
        └── 🔧 Failed Custom PDF Extractor (4)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6d3a286f-0f88-4b6f-bc85-59958f517592                                                                     │
│  Agent: VectorDB Manager                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/home/johnsonhk88/.local/lib/python3.10/site-packages/weave/trace/object_record.py:123: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  value = getattr(object, key)
/home/johnsonhk88/.local/lib/python3.10/site-packages/weave/trace/object_record.py:123: PydanticDeprecatedSince20: The `__fields_set__` attribute is deprecated, use `model_fields_set` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  value = getattr(object, key)


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1e4edb50-ccca-4d27-9c68-c01797a773cf                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

OPIK: Started logging traces to the "crewai-ai-bank-statement-document-analysis-project" project at http://localhost:5173/api/v1/session/redirect/projects/?trace_id=01973663-9f2b-748b-b985-57e3f6fe6675&path=aHR0cDovL2xvY2FsaG9zdDo1MTczL2FwaS8=.


In [61]:
# print(result.model_dump_json(
# 
Markdown(result.raw)

To store document content in the vector database based on the provided context and available tools, follow these steps:

1. **Prepare Your Directory**:
   - Ensure all bank statement PDF files are organized in a specific directory named "bank_statements".

2. **Use Custom PDF Extractor Tool**:
   - Action: `Custom PDF Extractor`
   - Input: `{ "pdf_dir": "path/to/your/bank_statements" }`

3. **Process the Output**:
   - After running the custom extractor, you will receive a list of dictionaries containing the extracted text from each PDF file.

4. **Example Content** (based on provided sample):
   ```json
   [
       {
           "content": "Issue Date:\nPeriod:\nAccount Activity\nDate\nDescription\nAmount\nBalance",
           "source": "../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf",
           "page_number": 1
       }
   ]
   ```

5. **Store Text in Vector Database**:
   - Use the `Vector DB Store tool` to store the extracted content.
   - Action: `Vector DB Store tool`
   - Input: 
     ```json
     {
         "doc": "Issue Date:\nPeriod:\nAccount Activity\nDate\nDescription\nAmount\nBalance\\n../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf\\n1"
     }
     ```

6. **Handling Errors**:
   - If the `Vector DB Store tool` encounters an error, consider using a custom approach with a different vector database tool like Pinecone or Weaviate.
   - Example for using Weaviate (assuming you have set up embeddings and the Weaviate API):
     ```python
     import weaviate

     client = weaviate.Client("http://localhost:8080")  # Adjust URL as needed

     class_obj = {
         "class": "BankStatement",
         "vectorProperty": { "propertyName": "content", "vectorDimension": 1536 }
     }

     if not client.schema.exists(class_obj):
         client.schema.create_class(class_obj)

     embedding = get_embedding("Issue Date:\nPeriod:\nAccount Activity\nDate\nDescription\nAmount\nBalance")  # Assume this function exists
     data_to_upsert = {
         "class": "BankStatement",
         "content": "Issue Date:\nPeriod:\nAccount Activity\nDate\nDescription\nAmount\nBalance",
         "source": "../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf",
         "page_number": 1,
         "vector": embedding
     }

     client.data_object.create(data_to_upsert, class_name="BankStatement")
     ```

By following these steps and ensuring the correct use of tools or custom approaches, you can effectively store vectorized financial data from PDF bank statements in a vector database.

In [ ]:
# print(result.raw)

Based on the extracted content from the first page of the PDF document, here is the JSON structure that represents the text content along with its source file path and the page number:

```json
[
    {
        "content": "Issue Date:\nPeriod:\nAccount Activity\nDate\nDescription\nAmount\nBalance.",
        "source": "../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf",
        "page_number": 0
    }
]
```


In [157]:
print(df.iloc[0]["metadata"])

{'producer': 'Microsoft: Print To PDF', 'creator': '', 'creationdate': '2020-07-03T16:22:11+08:00', 'source': '../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf', 'file_path': '../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': 'Bank Statement Template 2 - TemplateLab.xlsx', 'author': 'HFO Desktop', 'subject': '', 'keywords': '', 'moddate': '2020-07-03T19:36:51+08:00', 'trapped': '', 'modDate': "D:20200703193651+08'00'", 'creationDate': "D:20200703162211+08'00'", 'page': 0}


In [134]:
documents[0].page_content

'Issue Date:\nPeriod:\nAccount Activity\nDate\nPayment Type\nPaid In\nPaid Out\nBalance\nYour Account Statement\nDetail\nNote:\nPrint Form\nSave Form\nReset Form\n<Branch Name>\n231 Valley Farms Street \nSanta Monica, CA \nbickslowbank@domain.com\nmm/dd/yyyy\nmm/dd/yyyy to mm/dd/yyyy\n111-234-567-890  \nBit Manufacturing Ltd\n2450 Courage St, STE 108\nBrownsville, TX 78521\nBalance Brought Forward\n8,313.30\nmm/dd/yyyy Fast Payment\nAmazon\n132.30\n8,181.00\nmm/dd/yyyy BACS\neBAY Trading Co.\n515.22\n7,665.78\nmm/dd/yyyy Fast Payment\nMorrisons Petrol\n80.00\n7,585.78\nmm/dd/yyyy BACS\nBusiness Loan\n20,000.00\n27,585.78\nmm/dd/yyyy BACS\nJumes White Media\n2,416.85\n25,168.93\nmm/dd/yyyy Fast Payment\nATM High Street\n100.00\n25,068.93\nmm/dd/yyyy BACS\nAccorn Advertising Studios\n150.00\n24,918.93\nFast Payment\nmm/dd/yyyy\nMarriott Hotels\n177.00\n24,741.93\nmm/dd/yyyy Fast Payment\nAbelio Scotrail Ltd\n122.22\n24,619.71\nmm/dd/yyyy Fast Payment\nCheque 000234\n1,200.00\n23,419.71\n